In [1]:
import fepfilesetup as ffs

# Demonstration for the fepfilesetup package

1. Create a FEP campaign directory and copy this notebook within it

In [ ]:
campaign_name='FEP_campaign'

In [3]:
ffs.campaign_dir_setup(campaign_name)

Copy your 3 starting structures into the path:
FEP_campaign/00-compounds/rep_struc
Naming convention of starting representative structures is as follows:
OLDCOMP_repN_fNNNN.pdb
 OLDCOMP is the 3-letter code for the old compound
 repN is rep1, rep2 or rep3
 fNNNN is the frame number that was extracted from the OLDCOMP equilibrium MD


2. Define the compound from where the transformation will start from, and extract its PDB

In [4]:
oldcomp='DU1'

In [5]:
ffs.extract_oldcomp_pdb(oldcomp,repN='rep1',campaign_name=campaign_name)


3. Use Maestro to draw new compounds.
- Go to 00-compounds, and open {oldcomp}.pdb with Maestro
- Use Maestro's builder to create modified molecules
- Save them as pre_{newcomp}.mol2
- Open pre_{newcomp}.mol2 with VMD and save it as {newcomp}.pdb

In [5]:
newcomp_list=ffs.get_newcomp_list(oldcomp,campaign_name=campaign_name)

This is the list of new compounds that will be used going forward. 
Please recreate this variable if you wish to work with a smaller subset of the new compounds
['DU2']


4. Process the structures of the new compound. The code will do the following:
- Create a copy of the {oldmol}, for each replica, that contains only the common core atoms
- Later on, when creating the topology, the compound will be rebuilt mantaining the exact same coordinates for the common core, and using the optimized structure from the parametrization to reconstruct the softcore part of the new compouds

In [30]:
for newcomp in newcomp_list:
    ffs.generate_cutpdb(oldcomp,newcomp)

Cut PDBs for DU2 have been created


5. Compound parametrization section
- First, you must decide the type of parametrization between 'orca' or 'am1bcc'
- Second, the code will generate the input files to parametrize the compounds
- Third, and last, you must execute these scripts from the command line

Make sure AmberTools is up and running when executing the parametrization scripts.

If parametrizing with am1bcc, please include the charge of the compound.

In [9]:
param_type='orca'
ffs.setup_param(param_type,newcomp_list,campaign_name)

Go to:
FEP_campaign/01-mol_param/
and execute the script:
./param_orca_DU2.sh


In [12]:
param_type='am1bcc'
charge=0
execute_param=True

tempcomp_list=['DU1','DU2']
ffs.setup_param(param_type,tempcomp_list,campaign_name,charge,execute_param)


Compound DU2 parametrized


6. System setup - tleap
- This section will create a directory per compound in 02-leap, and subdirectories for each replica (rep1,rep2,rep3,unbound)
- Then, it will bring all necessary files to create the appropiate topology and restart file per compound, per replica
- Finally, it will execute tleap and output the .top and .rst files.

In [6]:
ffs.tleap_setup_organize_dirs(newcomp_list,oldcomp,campaign_name)
ffs.tleap_execute(newcomp_list,oldcomp,campaign_name)

Setting up: DU2-rep1
New molecule inserted at line 7299
Setting up: DU2-rep2
New molecule inserted at line 7299
Setting up: DU2-rep3
New molecule inserted at line 7299
Setting up: DU2-unbound
Executing DU2-rep1
Executing DU2-rep2
Executing DU2-rep3
Executing DU2-unbound


7. Setup directories and inputs for FEP transformations
- First, this section will create a directory for every compound in 03-md, with its replicas subdirectories
- Second, .top and .rst will be distributed to the relevant stdr and aces directories
- Third, it will further populate each subdirectory with all *.in files and the groupfile to run ACES replica exchange protocol
- Finally, it will generate a job submission script for each independent simulation (one in each subdirectory)
    - These job submission are organized so that only the stdr/mdsolv script must be submitted - it will then submit all replica's stdr jobs
    - All replica's stdr jobs will then submit their corresponding aces job

In [24]:
ffs.mdfep_organize_dirs(newcomp_list,campaign_name)

In [25]:
PROPS_stdr={
    'NTPR':5_000,       
    'NTC':2,           
    'NSTLIM':500_000,  #1ns
    'DT':0.002,        
    'GTI_BAT_SC':0     
}
PROPS_acesR={
    'NTPR':5_000,       
    'NTC':2,           
    'NSTLIM':2_000,  
    'DT':0.002,
    'NUMEXCHG':2_000,
    'GTI_BAT_SC':0     
}
PROPS_acesUB={
    'NTPR':5_000,       
    'NTC':2,           
    'NSTLIM':6_000, #3x as long for unbound
    'DT':0.002,
    'NUMEXCHG':2_000,
    'GTI_BAT_SC':0     
}
lambda_gap=[0.0,0.1,0.150,0.185,0.220,0.255,0.290,0.325,0.360,0.395,0.430,0.465,0.500,0.535,0.570,0.605,0.640,0.675,0.710,0.745,0.780,0.815,0.850,0.9,1.0]
for newcomp in newcomp_list:
    ffs.prepare_FEP_files(newcomp,oldcomp,campaign_name,lambda_gap,PROPS_stdr,PROPS_acesR,PROPS_acesUB,gap_ROUND=3)

In [31]:
ffs.distribute_submission_jobs(newcomp_list,campaign_name)

Copy the complete 03-md/DU2 directory into the HPC
In the HPC, go to:
 DU2/stdr/unbound/mdsolv
 and submit: 
qsub job_mdsolv_1.sh
